In [ ]:
pip install pyquery

In [ ]:
pip install tenacity

In [ ]:
pip install umls-api

In [ ]:
!pip install pyldavis --q

In [ ]:
pip install stanza

In [ ]:
import stanza
stanza.download('en',package = 'craft')

2021-07-31 23:16:43 INFO: Downloading these customized packages for language: en (English)...
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |
| pretrain  | craft   |

2021-07-31 23:16:43 INFO: File exists: /root/stanza_resources/en/tokenize/craft.pt.


2021-07-31 23:17:13 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
import requests
import json
from lxml.html import fromstring
import pyquery
from __future__ import print_function
import pandas as pd
import umls_api
import gensim
from gensim.models import Phrases
from gensim.corpora import Dictionary 

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
from gensim.parsing.preprocessing import strip_numeric,strip_multiple_whitespaces, strip_punctuation, strip_short, strip_tags

In [ ]:
import re
import numpy as np

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/MyDrive/bbc_1807_1906_novideos.csv',encoding = 'utf-8')
df.head

Mounted at /content/drive


<bound method NDFrame.head of      Month  ... Unnamed: 6
0     1901  ...        NaN
1     1901  ...        NaN
2     1901  ...        NaN
3     1901  ...        NaN
4     1901  ...        NaN
..     ...  ...        ...
707   1812  ...        NaN
708   1812  ...        NaN
709   1812  ...        NaN
710   1812  ...        NaN
711   1812  ...        NaN

[712 rows x 7 columns]>

In [ ]:
texts = list(df.loc[:, "text"].values)

In [ ]:
new_sw = ['people','say','says','said','could','one','would','take','want','have','she','he','give','told','know','but','health','month','day','year', 'minute','second','hundred','thousand','however']
stop_words = STOPWORDS.union(set(new_sw))

In [ ]:
pubmed_stoplist = ['a','about','again','all','almost', 'also','although','always',\
                   'among','an','and','another','any','are','as','at','be','because',\
                   'been','before','being','between','both','but','by','can','could',\
                   'did','do','does','done','due','during','each','either','enough', \
                   'especially','etc','for','found', 'from','further','had','is','it',\
                   'has','have','having','here','how','however','i','if','in','into',\
                   'its','itself','just','kg','km', 'made','mainly','make','may','mg',\
                   'might','ml','mm','most','mostly','must','nearly','neither','no',\
                   'nor','obtained','of','often','on','our','overall','perhaps','pmid',\
                   'quite','rather','really','regarding','seem','seen','several',\
                   'should','show','showed','shown','shows','significantly','since',\
                   'so','some','such','than','that','the','their','theirs','them',\
                   'then','there','therefore','these','they','this','those','through',\
                   'thus','to','upon','use','used','using','various','very', 'was','we',\
                   'were','what','when', 'which','while','with','within','without','would']

In [ ]:
stop_words = STOPWORDS.union(set(pubmed_stoplist))

In [ ]:
clean_article = []

for text in texts:
  #remove new line character and punctuations 
  clean = []
  text = strip_multiple_whitespaces(text)
  text = strip_numeric(text)
  text = strip_punctuation(text)
  text = strip_tags(text)
  text = strip_short(text,minsize=3)
  text = remove_stopwords(text)
  # lower case, de-accents.  Convert the texts into a list of tokens
  for token in gensim.utils.simple_preprocess(text, deacc=True):
      clean.append(token)
  clean_article.append(clean)

In [ ]:
nlp_craft = stanza.Pipeline('en',package = 'craft',processor = 'pos,tokenize,lemma',tokenize_pretokenized = True)

2021-07-31 23:18:17 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |

2021-07-31 23:18:17 INFO: Use device: gpu
2021-07-31 23:18:17 INFO: Loading: tokenize
2021-07-31 23:18:17 INFO: Loading: pos
2021-07-31 23:18:29 INFO: Loading: lemma
2021-07-31 23:18:29 INFO: Loading: depparse
2021-07-31 23:18:30 INFO: Done loading processors!


In [ ]:
doc_craft = nlp_craft (clean_article)
#4 min with GPU

In [ ]:
token_list = []
for i in range(len(doc_craft.sentences)):
  article_token_list = []
  for m in range(len(doc_craft.sentences[i].tokens)):
    article_token_list.append(doc_craft.sentences[i].words[m].lemma)
  token_list.append(article_token_list)


In [ ]:
print(token_list[0])

['the', 'cheese', 'addict', 'but', 'discover', 'recently', 'shake', 'core', 'barely', 'look', 'babybel', 'face', 'half', 'eat', 'halloumi', 'squeaklessly', 'lie', 'yellow', 'fridge', 'cheese', 'dream', 'shatter', 'for', 'lifetime', 'unfettered', 'devotion', 'possibly', 'cheese', 'foe', 'friend', 'that', 'addict', 'good', 'body', 'that', 'cheese', 'toast', 'these', 'question', 'begin', 'surface', 'couple', 'month', 'ago', 'begin', 'make', 'episode', 'new', 'podcast', 'bbc', 'all', 'hail', 'kale', 'look', 'dairy', 'scary', 'for', 'time', 'increasingly', 'question', 'logic', 'adult', 'drink', 'milk', 'while', 'milk', 'dairy', 'product', 'cheese', 'yoghurt', 'good', 'source', 'protein', 'calcium', 'form', 'healthy', 'balance', 'diet', 'michael', 'greger', 'nutritionfact', 'org', 'there', 'animal', 'planet', 'drink', 'milk', 'weaning', 'drink', 'milk', 'species', 'sense', 'reel', 'series', 'study', 'show', 'life', 'shortening', 'potential', 'drink', 'hormonal', 'stew', 'blithely', 'assume',

In [ ]:
bigram = gensim.models.Phrases(token_list, min_count=5, threshold=100) 
bigram_2 = gensim.models.phrases.Phraser(bigram)
bigram_words = [bigram_2[text] for text in token_list]

In [ ]:
print(bigram_words[0])

['the', 'cheese', 'addict', 'but', 'discover', 'recently', 'shake', 'core', 'barely', 'look', 'babybel', 'face', 'half', 'eat', 'halloumi', 'squeaklessly', 'lie', 'yellow', 'fridge', 'cheese', 'dream', 'shatter', 'for', 'lifetime', 'unfettered', 'devotion', 'possibly', 'cheese', 'foe', 'friend', 'that', 'addict', 'good', 'body', 'that', 'cheese', 'toast', 'these', 'question', 'begin', 'surface', 'couple', 'month', 'ago', 'begin', 'make', 'episode', 'new', 'podcast', 'bbc', 'all', 'hail', 'kale', 'look', 'dairy', 'scary', 'for', 'time', 'increasingly', 'question', 'logic', 'adult', 'drink', 'milk', 'while', 'milk', 'dairy', 'product', 'cheese', 'yoghurt', 'good', 'source', 'protein', 'calcium', 'form', 'healthy', 'balance', 'diet', 'michael', 'greger', 'nutritionfact', 'org', 'there', 'animal', 'planet', 'drink', 'milk', 'weaning', 'drink', 'milk', 'species', 'sense', 'reel', 'series', 'study', 'show', 'life', 'shortening', 'potential', 'drink', 'hormonal', 'stew', 'blithely', 'assume',

In [ ]:
apikey = "e22b627f-226b-420b-9de2-935e10ff3350"
service="http://umlsks.nlm.nih.gov"

In [ ]:
def get_granting_ticket():
     params = {'apikey': apikey}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post("https://utslogin.nlm.nih.gov/cas/v1/api-key",data=params,headers=h)
     response = fromstring(r.text)
     tgt = response.xpath('//form/@action')[0]
     return tgt

In [ ]:
def get_single_ticket(tgt):
     params = {'service': service}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post(tgt,data=params,headers=h)
     st = r.text
     return st

In [ ]:
cache = {} 

def umls_cache(keyword,query):
  if keyword in cache.keys():
    print("Fetching from cache ....")
    return cache[keyword]
  else:
    return get_umls(keyword, query)


def get_umls(keyword, query):
  version = "2021AA"
  print("Making a Query....")
  r = requests.get("https://uts-ws.nlm.nih.gov/rest/search/"+version,params=query)
  r.encoding = 'utf-8'
  items  = json.loads(r.text)
  umls = False

   try:
    cui = items["result"]["results"][0]["ui"]
    response = umls_api.API(api_key = 'e22b627f-226b-420b-9de2-935e10ff3350').get_cui(cui = cui)
    if response == 'Temporal Concept'or response == 'Qualitative Concept' or response == 'Quantitative Concept':
      umls = False 
    else:
      umls = True 
  except:
    umls = False

  cache[keyword] = umls
  return umls

In [ ]:
def term_type(string,tgt):
  ##generate a new service ticket for each page if needed
  ticket = get_single_ticket(tgt)
  query = {'string':string,'ticket':ticket}
  return umls_cache(string, query)  

In [ ]:
tgt = get_granting_ticket()

In [ ]:
umls_list = []

for article in bigram_words:
  article_umls_list =[]
  for word in article:
    umls = term_type(word,tgt)
    if umls is True:
      article_umls_list.append(word)
  umls_list.append(article_umls_list)

print(umls_list[0])

KeyError: ignored